## <a href="https://cursos.alura.com.br/course/langchain-python-ferramentas-llm-openai/task/156165"><b>Utilizando LCEL para criar um roteiro de viagens</b></a><br/>
Dar restart no kernel para limpar a memória. Não esquecer de liberar o scroll na última célula, para ver a saída completa.

In [ ]:
from langchain.prompts import ChatPromptTemplate # Uma interação de múltiplos prompts é um chat, por isso, vamos importar um ChatPromptTemplate

In [ ]:
%pip install -qr requirements.txt

#### <b>PASSO 1 - IMPORTS e CRIAÇÃO DA LLM</b>

In [ ]:
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env

load_dotenv() # CARREGANDO O ARQUIVO COM A OPENAI_KEY

llm = ChatOpenAI( # INSTANCIANDO A LLM
                    model="gpt-4.1-mini",
                    temperature=0.5,
                    # 1 - OBTENDO A API KEY POR MEIO DA VARIÁVEL DE AMBIENTE OPENAI_KEY. QUE VAI FICAR ARMAZENADA NO ARQUIVO .env.
                    # 2 - AINDA É NECESSÁRIO CARREGAR ESSE ARQUIVO. VER NA PRIMEIRA CÉLULA DO NOTEBOOK
                    api_key=getenv("OPENAI_KEY")
                )

#### <b>PASSO 2 - CRIANDO O <i>PROMPT TEMPLATE</i> E ASSOCIANDO UM PARSER A ELE</b></br> 

<b><ol><li>CRIANDO OS MODELOS</li></ol></b>

<ul><ul><li><b>CRIANDO O PARSER E ASSOCIANDO ELE AO MODELO DE CIDADE</b></li></ul></ul>

In [ ]:
from pydantic import Field,BaseModel # pydantic -> Biblioteca para validação de dados. Garante que os dados recebidos ou manipulados estejam no formato correto,
                                     # BaseModel -> Os modelos pydantic são classes que herdam BaseModel (https://docs-pydantic-dev.translate.goog/latest/concepts/models/?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc)
                                     # Modelos possuem campos como atributos.
                                     
                                     # Field -> Para personalizar os campos do modelo (https://docs-pydantic-dev.translate.goog/latest/concepts/fields/?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc)

class Destino(BaseModel): # A nossa classe vai estender a classe BaseModel, que terá dois campos a cidade e o motivo de visitá-la
    cidade: str = Field(description="cidade a visitar") # Descrição do campo. Apenas informativo
    motivo: str = Field(description="motivo pelo qual é interessante visitar") # Descrição do campo. Apenas informativo  

from langchain import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser # EXISTEM DIVERSOS OUTPUT PARSERS (https://python.langchain.com/docs/concepts/output_parsers/)

parseador = JsonOutputParser(pydantic_object=Destino) # DOCUMENTAÇÃO JsonOutputParser (https://python.langchain.com/docs/how_to/output_parser_json)

template_cidade = PromptTemplate(
                                    template="""Sugira uma cidade, dado o meu interesse por {interesse}.
                                    {formatacao_de_saida_da_ia}""", # AQUI TEMOS UMA VARÍAVEL PARCIAL. UTLIZAÇÃO DA TÉCNICA DE SHOTS PARA PROMPTS
                                    input_variables=["interesse"],
                                    # A VARÍAVEL PARCIAL É UM DICIONÁRIO. FUNCIONA COMO O SHOT
                                    partial_variables={"formatacao_de_saida_da_ia":parseador.get_format_instructions()}, # PASSANDO O PARSEADOR PARA A VARIÁVEL FORMATAÇÃO DE SAÍDA.
                                                                                                   
                                ) # INSTANCIANDO PromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>CRIANDO O MODELO PARA RESTAURANTES</b></li></ul></ul>

In [ ]:
template_restaurante = ChatPromptTemplate.from_template("Sugira restaurantes populares entre locais na {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>CRIANDO O MODELO CULTURAL</b></li></ul></ul>

In [ ]:
template_cultural = ChatPromptTemplate.from_template("Sugira atividades e locais culturais em {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

#### <b>PASSO 3 - CRIANDO AS CADEIAS</b></br> 

In [ ]:
from langchain.chains import LLMChain

cadeia_cidade = LLMChain(llm=llm,prompt=template_cidade)
cadeia_restaurante = LLMChain(llm=llm,prompt=template_restaurante)
cadeia_cultural = LLMChain(llm=llm,prompt=template_cultural)

<b><ul><li>CRIANDO A CADEIA GERAL (SimpleSequentialChain)</li></ul></b>

In [ ]:
from langchain.chains import SimpleSequentialChain

cadeia = SimpleSequentialChain(chains=[cadeia_cidade,cadeia_restaurante,cadeia_cultural])


#### <b>PASSO 4 - INVOCANDO A CADEIA GERAL</b>

In [ ]:
resposta = cadeia.invoke("praias") # DIFERENTE DO QUE ACONTECEU COM O PROMPT TEMPLATE, AQUI INVOCAMOS A CADEIA, E COMTÉM UM TEMPLATE, E ELA INVOCA A LLM
print(resposta)

#### <b>OBTENDO APENAS O CONTEÚDO DA MENSAGEM</b>

In [ ]:
print(resposta['output'])

<ul><li><b>USANDO O MODO DEBUG</b></li></ul>

In [ ]:
from langchain.globals import set_debug
set_debug(True)

resposta = cadeia.invoke(input="praias")
print(resposta)

<b>RESULTADO DOS PROMPTS</b>